# 3 Prompts（提示工程）

在 LangChain 框架中，**Prompts（提示工程）** 是连接用户意图与语言模型（LLM）的核心桥梁。它通过结构化的输入模板，精准引导模型生成符合预期的输出。与直接向 LLM 输入原始文本相比，精心设计的提示能显著提升模型性能 —— 这也是为什么 LangChain 将 Prompts 作为独立核心模块的原因。

Prompts 模块是 LangChain 中连接用户与模型的 "翻译官"，通过结构化模板将模糊需求转化为模型可理解的指令。核心要点：

- 根据模型类型选择PromptTemplate（文本模型）或ChatPromptTemplate（聊天模型）；

- 利用变量替换、部分绑定、格式约束等功能提升提示灵活性；

- 复杂场景通过提示组合实现模块化管理。

## 一、Prompts 的核心价值：为什么需要提示工程？

语言模型本身是 "通用型" 的，需要通过提示来约束其行为。例如：

- 同样问 "介绍 LangChain"，提示 "用 300 字学术风格" 和 "用 3 句话通俗解释" 会得到完全不同的结果；

- 缺乏格式约束时，模型可能返回杂乱的文本，而提示 "以 JSON 格式输出关键词" 能直接得到结构化数据。

LangChain 的 Prompts 模块解决了三大问题：

1. **标准化输入**：通过模板固定提示结构，避免重复编写相同指令；

2. **动态变量替换**：支持在模板中插入变量（如用户输入、上下文信息）；

3. **多模态适配**：针对不同类型模型（文本模型 / 聊天模型）提供专用模板。

## 二、Prompts 核心组件与类型

LangChain 官网明确将 Prompt Templates 分为三类：`String PromptTemplates`（单字符串模板）、`ChatPromptTemplates`（多消息模板）、`MessagesPlaceholder`（消息列表占位符）。以下按官网示例逐一讲解，确保代码零错误。

|提示类型|	适用模型|	核心特点|
|--|--|--|
|`String PromptTemplates`|	用于生成单一字符串的模板|	输入是 “变量字典”，输出是`PromptValue`对象（可转字符串）。|
|`ChatPromptTemplates`|用于生成多轮消息列表的模板，适配「聊天模型」（如 GPT-3.5/4、Claude）|输入是 “变量字典”，输出是含多角色消息的PromptValue对象（可转BaseMessage列表）|
|`MessagesPlaceholder`|用于在 ChatPromptTemplate 中插入动态消息列表（如对话历史、外部消息记录），解决 “固定模板 + 动态多轮消息” 的组合问题。|多轮对话中，将历史消息（人类 + AI）嵌入到固定模板中，避免手动拼接消息列表|


## 三、实践案例

### 案例1：单字符串模板（String PromptTemplates）

In [9]:
from dotenv import load_dotenv
import os
from langchain_core.prompts import PromptTemplate

load_dotenv()

# 1. 创建字符串模板（两种方式）
# 方式1：直接指定模板字符串（推荐，代码更简单）
prompt_template = PromptTemplate.from_template("""
请用3句话解释{concept}, 目标读者是{audience}
""")

# 方式2：显示指定 input_variables
# prompt_template = PromptTemplate(
#     template="请用3句话解释{concept}, 目标读者是{audience}",
#     input_variables=["concept", "audience"] # 必须与模板变量名完全一致
# )

# 2. 传入变量生成PromptValue
prompt_value = prompt_template.invoke({
    "concept": "LangChain的PromptTemplate",
    "audience": "Python初学者"
})

# 3. 适配不同的模型：转为字符串（给纯文本模型）或查看原始PromptValue
print("=== PromptValue对象信息 ===")
print(f"类型：{type(prompt_value)}")
print(f"转字符串（给纯文本模型）：{prompt_value.to_string()}")

=== PromptValue对象信息 ===
类型：<class 'langchain_core.prompt_values.StringPromptValue'>
转字符串（给纯文本模型）：
请用3句话解释LangChain的PromptTemplate, 目标读者是Python初学者



### 案例2：多消息模板（ChatPromptTemplates）

In [24]:
from dotenv import load_dotenv
import os
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
import time

load_dotenv()

chat_prompt_template = ChatPromptTemplate([
    # 角色1：system ———— 定义模型身份
    {"role":"system", "content":"你是'{domain}'领域的专业助手，你负责提供准确且精要的专业描述，回答不超过200字。"},
    # 角色2：user ———— 用户动态输入
    {"role":"human", "content":"我的问题是：{question}"},
])

# 传入变量生成PromptValue
prompt_value = chat_prompt_template.invoke({
    "domain": "Python 编程",
    "question": "LangChain的ChatPromptTemplate和PromptTemplate有什么区别？",
})

messages = prompt_value.to_messages()
for msg in messages:
    print(f"角色：{msg.type}，内容：{msg.content}")

llm = ChatOpenAI(
    api_key=os.getenv("UIUIAPI_API_KEY"),
    base_url=os.getenv("UIUIAPI_BASE_URL"),
    model="gpt-3.5-turbo",
    temperature=0.8,
)
for chunk in llm.stream(messages):
    for ch in chunk.content:
        print(ch, end='', flush=True)
        time.sleep(0.05)


角色：system，内容：你是'Python 编程'领域的专业助手，你负责提供准确且精要的专业描述，回答不超过200字。
角色：human，内容：我的问题是：LangChain的ChatPromptTemplate和PromptTemplate有什么区别？
在LangChain中，ChatPromptTemplate用于生成对话式文本，而PromptTemplate则用于生成非对话式文本。ChatPromptTemplate更适合用于生成对话流程，包含用户和系统之间的交互，而PromptTemplate更适合用于生成单一文本。因此，区别主要在于用途和生成文本的形式。

### 案例3：消息列表占位符(MessagesPlaceholder)

In [26]:
from dotenv import load_dotenv
import os
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.messages import HumanMessage, AIMessage
from langchain_openai import ChatOpenAI

load_dotenv()

# 1. 创建含占位符的聊天模板
chat_prompt_template = ChatPromptTemplate([
    # 固定消息
    {"role":"system", "content": "你是记忆助手，会基于历史对话回答新问题，不会编造信息。"},
    # 动态消息占位符：外部传入的消息列表将插入到这里
    MessagesPlaceholder(variable_name="msgs")
])

# 2. 准备外部消息列表（如对话历史）——官网示例格式
conversation_history = [
    HumanMessage(content="LangChain是什么？"),
    AIMessage(content="LangChain是构建LLM应用的框架，核心是组件化和可组合性。"),
    HumanMessage(content="它的核心模块有哪些？")  # 最新问题
]

prompt_value = chat_prompt_template.invoke({
    "msgs": conversation_history
})

# 查看最终消息列表（固定系统消息 + 动态历史消息）
print("=== 最终消息列表（系统消息+历史对话） ===")
messages = prompt_value.to_messages()
for i, msg in enumerate(messages, 1):
    print(f"{i}. 角色：{msg.type}，内容：{msg.content}")

for chunk in llm.stream(messages):
    for char in chunk.content:
        print(char, end='', flush=True)
        time.sleep(0.05)


=== 最终消息列表（系统消息+历史对话） ===
1. 角色：system，内容：你是记忆助手，会基于历史对话回答新问题，不会编造信息。
2. 角色：human，内容：LangChain是什么？
3. 角色：ai，内容：LangChain是构建LLM应用的框架，核心是组件化和可组合性。
4. 角色：human，内容：它的核心模块有哪些？
LangChain的核心模块包括语言处理模块、链式编程模块和应用适配模块。语言处理模块负责处理自然语言输入和输出，链式编程模块负责实现链式调用的编程风格，应用适配模块负责将LangChain框架与具体应用场景进行适配。这些模块共同工作，构建了LangChain的整体架构。

**官网更推荐的写法：避免导入MessagesPlaceholder**

In [27]:
chat_prompt_template = ChatPromptTemplate([
    {"role":"system", "content": "你是记忆助手，会基于历史对话回答新问题，不会编造信息。"},
    {"role":"placeholder", "content": "{msgs}"}
])

conversation_history = [
    HumanMessage(content="LangChain是什么？"),
    AIMessage(content="LangChain是构建LLM应用的框架，核心是组件化和可组合性。"),
    HumanMessage(content="它的核心模块有哪些？")  # 最新问题
]

prompt_value = chat_prompt_template.invoke({
    "msgs": conversation_history
})

# 查看最终消息列表（固定系统消息 + 动态历史消息）
print("=== 最终消息列表（系统消息+历史对话） ===")
messages = prompt_value.to_messages()
for i, msg in enumerate(messages, 1):
    print(f"{i}. 角色：{msg.type}，内容：{msg.content}")

for chunk in llm.stream(messages):
    for char in chunk.content:
        print(char, end='', flush=True)
        time.sleep(0.05)

=== 最终消息列表（系统消息+历史对话） ===
1. 角色：system，内容：你是记忆助手，会基于历史对话回答新问题，不会编造信息。
2. 角色：human，内容：LangChain是什么？
3. 角色：ai，内容：LangChain是构建LLM应用的框架，核心是组件化和可组合性。
4. 角色：human，内容：它的核心模块有哪些？
LangChain的核心模块包括：

1. 语言模块：处理自然语言理解和生成
2. 知识模块：管理知识图谱和相关知识库
3. 编程模块：支持编写和执行LLM应用的代码
4. 控制模块：管理应用的流程和交互逻辑
5. 学习模块：支持模型的在线学习和优化

## 四、提示工程最佳实践

1. **明确角色与任务**：在系统消息中清晰定义模型角色（如：你是数据分析师）和任务目标

2. **提供示例（少样本提示）**：复杂任务时加入示例（如“输出格式如下：示例1 ...”）

3. **控制输出长度**：明确约束（如“不超过3句话”），避免模型输出过长或过短

4. **使用分隔符**：用、===等分隔不同内容

5. **逐步优化**：根据输出不断重新调整提示词（如补充：“不要使用专业术语”）

## 五、常见问题与解决方案

|问题|	解决方案|
|--|--|
|模型输出格式混乱|	用JsonOutputParser强制格式，或在提示中加入示例|
|模型忽略部分指令|	关键要求放在提示开头，或用加粗（**）强调|
|提示过长超出 Token 限制|	拆分提示为子模板，或使用partial()绑定固定内容|